# Setting

In [3]:
# Parameter Setting
import pandas as pd
import torch

folder_name = 'Research-(D5) Synthesized input model'

time_start = '2016-01-01T00:00:00'
time_end = '2019-12-31T00:00:00'

time_start = pd.to_datetime('2016-01-01T00:00:00')
time_end = pd.to_datetime('2019-12-31T00:00:00')

config = {
    'seed': 42,
    'shuffle':False,
}

# print(len(feature)-2)

company_list = [
    "Information Technology",
    "Health Care",
    "Financials",
    "Industrials",
    "Consumer Discretionary",
    "Energy",
    "Materials",
    "Communication Services",
    "Utilities",
    "Real Estate",
    "Consumer Staples"
]

process_id = 0  #26

company_name = company_list[process_id]
config_2 = {'input_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/data/2_'+company_name+'_for_model.csv',
            'save_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model.ckpt',
            }

In [4]:
# Google
from google.colab import drive
drive.mount('/content/drive')

# pip installation
!pip install yfinance

import yfinance as yf

# Basic
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# others
from datetime import datetime, timedelta


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed for reproducibility
same_seed(config['seed'])


# Benchmark: Mean of excess return

### (1) Get data

In [6]:
def datetime_func(df):
    # Convert the index to a datetime column
    df['datetime'] = df.index

    # Remove the time zone information
    df['datetime'] = pd.to_datetime(df['datetime']).dt.tz_localize(None).dt.date

    return df

# (1) Today Trend
def day_return(df, n, id):
  # Calculate Numerical
  df['today_return'] = (df['Close'] - df['Open']) / df['Open']

  # Calculate categorical
  df['today_return_cate'] = (df['Close'] - df['Open']).apply(lambda x: 1 if x >= 0 else 0)

  # Convert numerical trend to categorical
  df['today_return_cate'] = df['today_return_cate'].astype('category')

  for days in range(1, n+1):
    df[f'{id}_{days}_day_return'] = (df['Close'].shift(-days) / df['Open'].shift(-1) - 1)
  # for days in range(1, n+1):
  #   df[f'{id}_{days}_day_return_cate'] = df[f'{days}_day_return'].apply(lambda x: 1 if x >= 0 else 0)

  return df


In [7]:
# ids = ["^DJI", "^GSPC", "^NDX", "^IXIC", "^SOX", '^NYA', 'XLK']  # DJSI
ids = ["^GSPC", 'XLK']  # DJSI

df_list = []

for id in ids:
  dji = yf.Ticker(id)
  df = dji.history(period='Max')
  df = datetime_func(df)
  df = day_return(df, n=1, id=id)
  df[(str(id))] = (df['Close']-df['Open']) / df['Open']
  # df = df[[(str(id)), f'{id}_1_day_return', 'datetime']]
  df_list.append(df)

# Merge dataframes
df_index = df_list[0]
for df in df_list[1:]:
  df_index = pd.merge(df_index, df, on='datetime', how='inner')

df_index

,Open_x,High_x,Low_x,Close_x,Volume_x,Dividends_x,Stock Splits_x,datetime,today_return_x,today_return_cate_x,...,Low_y,Close_y,Volume_y,Dividends_y,Stock Splits_y,Capital Gains,today_return_y,today_return_cate_y,XLK_1_day_return,XLK
0,1202.839966,1209.219971,1192.719971,1203.569946,680500000,0.0,0.0,1998-12-22,0.000607,1,...,24.042181,24.243124,300500,0.0,0.0,0.0,-0.011090,0,0.001908,-0.011090
1,1203.569946,1229.890015,1203.569946,1228.540039,697500000,0.0,0.0,1998-12-23,0.020747,1,...,24.325867,24.822313,338300,0.0,0.0,0.0,0.001908,1,-0.003810,0.001908
2,1228.540039,1229.719971,1224.849976,1226.270020,246980000,0.0,0.0,1998-12-24,-0.001848,0,...,24.633195,24.727757,243700,0.0,0.0,0.0,-0.003810,0,-0.012241,-0.003810
3,1226.270020,1231.520020,1221.170044,1225.489990,531560000,0.0,0.0,1998-12-28,-0.000636,0,...,24.656839,24.798681,309300,0.0,0.0,0.0,-0.012241,0,-0.001898,-0.012241
4,1225.489990,1241.859985,1220.780029,1241.810059,586490000,0.0,0.0,1998-12-29,0.013317,1,...,24.585917,24.869600,123800,0.0,0.0,0.0,-0.001898,0,0.001912,-0.001898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6249,4235.790039,4259.379883,4219.430176,4247.680176,3821820000,0.0,0.0,2023-10-24,0.002807,1,...,163.479996,165.259995,5590200,0.0,0.0,0.0,0.002913,1,-0.016501,0.002913
6250,4232.419922,4232.419922,4181.419922,4186.770020,3869370000,0.0,0.0,2023-10-25,-0.010786,0,...,162.820007,163.309998,7764800,0.0,0.0,0.0,-0.016501,0,-0.020844,-0.016501
6251,4175.990234,4183.600098,4127.899902,4137.229980,4277640000,0.0,0.0,2023-10-26,-0.009282,0,...,159.500000,160.190002,10323200,0.0,0.0,0.0,-0.020844,0,-0.000310,-0.020844
6252,4152.930176,4156.700195,4103.779785,4117.370117,4019500000,0.0,0.0,2023-10-27,-0.008563,0,...,160.410004,161.119995,8114100,0.0,0.0,0.0,-0.000310,0,0.005433,-0.000310


### (2) excess return

In [8]:
# n_da_return, excess_return are "targets"

for id in ids:
  df_index[f'excess_return_{id}'] = df_index['XLK_1_day_return'] - df_index[f'{id}_1_day_return']
  df_index[f'excess_return_{id}_cate'] = df_index[f'excess_return_{id}'].apply(lambda x: 1 if x >= 0 else 0)

df = df_index
del df_index

In [9]:
df

,Open_x,High_x,Low_x,Close_x,Volume_x,Dividends_x,Stock Splits_x,datetime,today_return_x,today_return_cate_x,...,Stock Splits_y,Capital Gains,today_return_y,today_return_cate_y,XLK_1_day_return,XLK,excess_return_^GSPC,excess_return_^GSPC_cate,excess_return_XLK,excess_return_XLK_cate
0,1202.839966,1209.219971,1192.719971,1203.569946,680500000,0.0,0.0,1998-12-22,0.000607,1,...,0.0,0.0,-0.011090,0,0.001908,-0.011090,-0.018838,0,0.0,1
1,1203.569946,1229.890015,1203.569946,1228.540039,697500000,0.0,0.0,1998-12-23,0.020747,1,...,0.0,0.0,0.001908,1,-0.003810,0.001908,-0.001962,0,0.0,1
2,1228.540039,1229.719971,1224.849976,1226.270020,246980000,0.0,0.0,1998-12-24,-0.001848,0,...,0.0,0.0,-0.003810,0,-0.012241,-0.003810,-0.011605,0,0.0,1
3,1226.270020,1231.520020,1221.170044,1225.489990,531560000,0.0,0.0,1998-12-28,-0.000636,0,...,0.0,0.0,-0.012241,0,-0.001898,-0.012241,-0.015215,0,0.0,1
4,1225.489990,1241.859985,1220.780029,1241.810059,586490000,0.0,0.0,1998-12-29,0.013317,1,...,0.0,0.0,-0.001898,0,0.001912,-0.001898,0.009868,1,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6249,4235.790039,4259.379883,4219.430176,4247.680176,3821820000,0.0,0.0,2023-10-24,0.002807,1,...,0.0,0.0,0.002913,1,-0.016501,0.002913,-0.005715,0,0.0,1
6250,4232.419922,4232.419922,4181.419922,4186.770020,3869370000,0.0,0.0,2023-10-25,-0.010786,0,...,0.0,0.0,-0.016501,0,-0.020844,-0.016501,-0.011562,0,0.0,1
6251,4175.990234,4183.600098,4127.899902,4137.229980,4277640000,0.0,0.0,2023-10-26,-0.009282,0,...,0.0,0.0,-0.020844,0,-0.000310,-0.020844,0.008252,1,0.0,1
6252,4152.930176,4156.700195,4103.779785,4117.370117,4019500000,0.0,0.0,2023-10-27,-0.008563,0,...,0.0,0.0,-0.000310,0,0.005433,-0.000310,-0.001193,0,0.0,1


### (3) SMA

In [10]:
# 计算5日SMA
period_list = [
    1,
    2,
    3,
    4,
    5,
    10,
    20,
    120,
    240,
    720,
    1200,
    2400,
    4800
]

sma_period = 240
df['SMA'] = df['excess_return_^GSPC'].rolling(window=sma_period).mean()
# df['SMA'] = 1
df['SMA']

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
6249    0.000989
6250    0.000915
6251    0.000905
6252    0.000881
6253         NaN
Name: SMA, Length: 6254, dtype: float64

In [11]:
df['SMA'] = df['SMA'].shift(1)
df['SMA']

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
6249    0.001011
6250    0.000989
6251    0.000915
6252    0.000905
6253    0.000881
Name: SMA, Length: 6254, dtype: float64

In [12]:
df['pred'] = df['SMA'] - df['^GSPC_1_day_return']
df

,Open_x,High_x,Low_x,Close_x,Volume_x,Dividends_x,Stock Splits_x,datetime,today_return_x,today_return_cate_x,...,today_return_y,today_return_cate_y,XLK_1_day_return,XLK,excess_return_^GSPC,excess_return_^GSPC_cate,excess_return_XLK,excess_return_XLK_cate,SMA,pred
0,1202.839966,1209.219971,1192.719971,1203.569946,680500000,0.0,0.0,1998-12-22,0.000607,1,...,-0.011090,0,0.001908,-0.011090,-0.018838,0,0.0,1,NaN,NaN
1,1203.569946,1229.890015,1203.569946,1228.540039,697500000,0.0,0.0,1998-12-23,0.020747,1,...,0.001908,1,-0.003810,0.001908,-0.001962,0,0.0,1,NaN,NaN
2,1228.540039,1229.719971,1224.849976,1226.270020,246980000,0.0,0.0,1998-12-24,-0.001848,0,...,-0.003810,0,-0.012241,-0.003810,-0.011605,0,0.0,1,NaN,NaN
3,1226.270020,1231.520020,1221.170044,1225.489990,531560000,0.0,0.0,1998-12-28,-0.000636,0,...,-0.012241,0,-0.001898,-0.012241,-0.015215,0,0.0,1,NaN,NaN
4,1225.489990,1241.859985,1220.780029,1241.810059,586490000,0.0,0.0,1998-12-29,0.013317,1,...,-0.001898,0,0.001912,-0.001898,0.009868,1,0.0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6249,4235.790039,4259.379883,4219.430176,4247.680176,3821820000,0.0,0.0,2023-10-24,0.002807,1,...,0.002913,1,-0.016501,0.002913,-0.005715,0,0.0,1,0.001011,0.011796
6250,4232.419922,4232.419922,4181.419922,4186.770020,3869370000,0.0,0.0,2023-10-25,-0.010786,0,...,-0.016501,0,-0.020844,-0.016501,-0.011562,0,0.0,1,0.000989,0.010271
6251,4175.990234,4183.600098,4127.899902,4137.229980,4277640000,0.0,0.0,2023-10-26,-0.009282,0,...,-0.020844,0,-0.000310,-0.020844,0.008252,1,0.0,1,0.000915,0.009477
6252,4152.930176,4156.700195,4103.779785,4117.370117,4019500000,0.0,0.0,2023-10-27,-0.008563,0,...,-0.000310,0,0.005433,-0.000310,-0.001193,0,0.0,1,0.000905,-0.005722


### (4) Categorize

In [13]:
  df['pred_cate'] = df['pred'].apply(lambda x: 1 if x >= 0 else 0)
  df['pred_cate'] = df['pred_cate'].astype('category')

# train_test_split

## (1) Time Period

In [14]:
# We use index to filter for time periods
df = df[(df['datetime']> time_start) & (df['datetime'] < time_end)]

# Drop datetime after using it
df.drop(columns=['datetime'], inplace=True)
df.shape

<ipython-input-14-c71406b1a572>:2: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df = df[(df['datetime']> time_start) & (df['datetime'] < time_end)]
<ipython-input-14-c71406b1a572>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['datetime'], inplace=True)


(1005, 30)

## (2) Split

In [15]:
# 1. Set up X, y
X = df['pred_cate']
y = df['excess_return_^GSPC_cate']

In [16]:
# 2. train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=config['seed'], shuffle=config['shuffle'])
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=config['seed'], shuffle=config['shuffle'])

## (3) Check data

In [17]:
def calculate_class_stats(y):
    class_counts = y.value_counts()
    total_samples = len(y)
    class_ratios = class_counts / total_samples
    return class_counts, class_ratios

# 計算類別數量和比例
train_class_counts, train_class_ratios = calculate_class_stats(y_train)
val_class_counts, val_class_ratios = calculate_class_stats(y_val)
test_class_counts, test_class_ratios = calculate_class_stats(y_test)

# 創建包含數量和比例的 DataFrame
class_stats = pd.DataFrame({
    'Train Count': train_class_counts,
    'Train Ratio': train_class_ratios,
    'Validation Count': val_class_counts,
    'Validation Ratio': val_class_ratios,
    'Test Count': test_class_counts,
    'Test Ratio': test_class_ratios
})

# 打印 DataFrame
print(class_stats)


   Train Count  Train Ratio  Validation Count  Validation Ratio  Test Count  \
1          322     0.533997               113          0.562189         113   
0          281     0.466003                88          0.437811          88   

   Test Ratio  
1    0.562189  
0    0.437811  


In [18]:
# Time period
print('Time Period')
print('From:', time_start)
print('To:', time_end, '\n')

# Sample size
print('Sample size:', X.shape[0])
# print('Feature:', X.columns, '\n')
print('Target:', y.name, '\n')
print('Train: Val: Test =', X_train.shape[0]/X.shape[0], X_test.shape[0]/X.shape[0], X_val.shape[0]/X.shape[0],
      X_train.shape[0], X_test.shape[0], X_val.shape[0])

Time Period
From: 2016-01-01 00:00:00
To: 2019-12-31 00:00:00 

Sample size: 1005
Target: excess_return_^GSPC_cate 

Train: Val: Test = 0.6 0.2 0.2 603 201 201


## Prediction

In [22]:
def report(text, y, pred):
  print('=====================================================================================================================')
  print(f'{text} Result:')
  print(classification_report(y, pred))
  print(confusion_matrix(y, pred), '\n')
  print('macro_f1: ', f1_score(y, pred, average='macro'))
  print('weighted_f1: ', f1_score(y, pred, average='weighted'))

def model(text, X, y):
  accuracy = accuracy_score(X, y)
  print(accuracy)

model('Train     ', X_train, y_train)
model('Validation', X_val, y_val)
model('Test      ', X_test, y_test)

report("Train", y_train, X_train)
report("Val", y_val, X_val)
report("Test", y_test, X_test)

0.5058043117744611
0.3681592039800995
0.47761194029850745
Train Result:
              precision    recall  f1-score   support

           0       0.47      0.52      0.50       281
           1       0.54      0.49      0.51       322

    accuracy                           0.51       603
   macro avg       0.51      0.51      0.51       603
weighted avg       0.51      0.51      0.51       603

[[147 134]
 [164 158]] 

macro_f1:  0.5056398010388239
weighted_f1:  0.5062529774171078
Val Result:
              precision    recall  f1-score   support

           0       0.32      0.39      0.35        88
           1       0.43      0.35      0.39       113

    accuracy                           0.37       201
   macro avg       0.37      0.37      0.37       201
weighted avg       0.38      0.37      0.37       201

[[34 54]
 [73 40]] 

macro_f1:  0.3675956893348198
weighted_f1:  0.3699436670234854
Test Result:
              precision    recall  f1-score   support

           0       0.4